In [ ]:
import psycopg2
conn = psycopg2.connect("host= dbname= user= port= password= ")
cur = conn.cursor()
cur.execute("SELECT * FROM records WHERE DATE(source_created_at + INTERVAL '3 hour') = CURRENT_DATE - INTERVAL '1 day'")
list_from_db1 = cur.fetchall()
cur.execute("SELECT * FROM records WHERE DATE(source_created_at + INTERVAL '3 hour') = CURRENT_DATE - INTERVAL '2 day'")
list_from_db2 = cur.fetchall()
cur.execute("SELECT id, name FROM customers")
list_from_db3 = cur.fetchall()

In [ ]:
import pandas as pd
df1 = pd.DataFrame(list_from_db1, columns=["id", "customer_id", "source", "category", "source_id", "additional_data", "text", "sentiment", "processed", "objects", "source_created_at", "created_at", "updated_at", "coverage", "coverage_data", "author_processed"])
df2 = pd.DataFrame(list_from_db2, columns=["id", "customer_id", "source", "category", "source_id", "additional_data", "text", "sentiment", "processed", "objects", "source_created_at", "created_at", "updated_at", "coverage", "coverage_data", "author_processed"])

In [ ]:
df1cust = df1
df2cust = df2

In [ ]:
customers_map = {}
customers = ['All']
i=0
while i<len(list_from_db3):
    x = list_from_db3[i]
    customers_map[x[1]] = x[0]
    customers.append(x[1])
    i=i+1

In [ ]:
from transliterate import translit, get_available_language_codes # транслит
def traslit(x):
    return translit(x, 'ru', reversed=True)

In [ ]:
def sign(x):
    if x>0:
        return '+'+str(x)
    else:
        return str(x)

In [ ]:
def df3_calculate(df1,df2):
    df3 = pd.DataFrame({'Customers':customers})
    def count_of_records_for_yesterday(x):
        if x == 'All':
            return len(df1)
        else:
            return len(df1[(df1.customer_id == customers_map[x])&(df1.processed == 1)])
    df3['Count of records for yesterday'] = df3['Customers'].map(count_of_records_for_yesterday)
    def count_of_records_for_2_days_ago(x):
        if x == 'All':
            return len(df2)
        else:
            return len(df2[(df2.customer_id == customers_map[x])&(df2.processed == 1)])
    df3['Count of records for 2 days ago'] = df3['Customers'].map(count_of_records_for_2_days_ago)
    def count_of_records_dynamics(x):
        if x == 'All':
            if len(df1) == len(df2):
                return '0%'
            elif len(df2) == 0:
                return '+100%'
            else:
                return sign(round((len(df1)/len(df2)-1)*100))+'%'
        else:
            if len(df1[(df1.customer_id == customers_map[x])&(df1.processed == 1)]) == len(df2[(df2.customer_id == customers_map[x])&(df2.processed == 1)]):
                return '0%'
            elif len(df2[(df2.customer_id == customers_map[x])&(df2.processed == 1)]) == 0:
                return '+100%'
            else:
                return sign(round((len(df1[(df1.customer_id == customers_map[x])&(df1.processed == 1)])/len(df2[(df2.customer_id == customers_map[x])&(df2.processed == 1)])-1)*100))+'%'
    df3['Dynamics'] = df3['Customers'].map(count_of_records_dynamics)
    df3['Customers'] = df3['Customers'].map(traslit)
    return df3

In [ ]:
def df4_calculate(df1,df2):
    processed_map = {'Error in processing':-2, 'In process':-1, 'Queue up':0, 'The right topic':1, 'Wrong topic':2, 'Spam':3}
    processed = ['Error in processing', 'In process', 'Queue up', 'The right topic', 'Wrong topic', 'Spam']
    df4 = pd.DataFrame({'Processed':processed})
    def processed_for_yesterday(x):
        return len(df1[df1.processed == processed_map[x]])
    df4['Count of records for yesterday'] = df4['Processed'].map(processed_for_yesterday)
    def processed_for_2_days_ago(x):
        return len(df2[df2.processed == processed_map[x]])
    df4['Count of records for 2 days ago'] = df4['Processed'].map(processed_for_2_days_ago)
    def processed_dynamics(x):
        if len(df1[df1.processed == processed_map[x]]) == len(df2[df2.processed == processed_map[x]]):
            return '0%'
        elif len(df2[df2.processed == processed_map[x]]) == 0:
            return '+100%'
        else:
            return sign(round((len(df1[df1.processed == processed_map[x]])/len(df2[df2.processed == processed_map[x]])-1)*100))+'%'
    df4['Dynamics'] = df4['Processed'].map(processed_dynamics)
    return df4

In [ ]:
def df5_calculate(df1cust,df2cust,customer):
    sentiment_map = {'Positive':0, 'Negative':1, 'Neutral':2, 'Undefined':3}
    sentiment = ['Positive', 'Negative', 'Neutral', 'Undefined']
    sentiment.append(customer)
    df5 = pd.DataFrame({'Sentiment':sentiment})
    def sentiment_for_yesterday(x):
        if x == customer:
            return len(df1cust[df1cust.processed == 1])
        else:
            return len(df1cust[(df1cust.sentiment == sentiment_map[x])&(df1cust.processed == 1)])    
    df5['Count of records for yesterday'] = df5['Sentiment'].map(sentiment_for_yesterday)
    def sentiment_for_2_days_ago(x):
        if x == customer:
            return len(df2cust[df2cust.processed == 1])
        else:
            return len(df2cust[(df2cust.sentiment == sentiment_map[x])&(df2cust.processed == 1)])
    df5['Count of records for 2 days ago'] = df5['Sentiment'].map(sentiment_for_2_days_ago)
    def sentiment_dynamics(x):
        if x == customer:
            if len(df1cust[df1cust.processed == 1]) == len(df2cust[df2cust.processed == 1]):
                return '0%'
            elif len(df2cust[df2cust.processed == 1]) == 0:
                return '+100%'
            else:
                return sign(round((len(df1cust[df1cust.processed == 1])/len(df2cust[df2cust.processed == 1])-1)*100))+'%'
        else:
            if len(df1cust[(df1cust.sentiment == sentiment_map[x])&(df1cust.processed == 1)]) == len(df2cust[(df2cust.sentiment == sentiment_map[x])&(df2cust.processed == 1)]):
                return '0%'
            elif len(df2cust[(df2cust.sentiment == sentiment_map[x])&(df2cust.processed == 1)]) == 0:
                 return '+100%'
            else:
                return sign(round((len(df1cust[(df1cust.sentiment == sentiment_map[x])&(df1cust.processed == 1)])/len(df2cust[(df2cust.sentiment == sentiment_map[x])&(df2cust.processed == 1)])-1)*100))+'%'
    df5['Dynamics'] = df5['Sentiment'].map(sentiment_dynamics)   
    return df5

In [ ]:
def df6_calculate(df1cust,customer):
    x = df1cust['objects'].values.tolist()
    y = []
    for row in x:
        for elem in row:
            y.append(elem)
    i = 0
    z = {}
    while i<len(y):
        z[y[i]] = y.count(y[i])
        i = i+1
    z[customer] = 100000
    df6 = pd.DataFrame(list(z.items()),columns=['Objects','Count'])
    df6['Objects'] = df6['Objects'].map(traslit)
    df6 = df6.sort_values(ascending=False,by=['Count']).head(10)
    return df6['Objects']

In [ ]:
def df7_calculate(df1,df2):
    source_map = {'VK':0, 'Twitter':1, 'YouTube':3}
    source = ['VK', 'Twitter', 'YouTube']
    df7 = pd.DataFrame({'Source':source})
    def source_for_yesterday(x):
        return len(df1[df1.source == source_map[x]])
    df7['Count of records for yesterday'] = df7['Source'].map(source_for_yesterday)
    def source_for_2_days_ago(x):
        return len(df2[df2.source == source_map[x]])
    df7['Count of records for 2 days ago'] = df7['Source'].map(source_for_2_days_ago)
    def source_dynamics(x):
        if len(df1[df1.source == source_map[x]]) == len(df2[df2.source == source_map[x]]):
            return '0%'
        elif len(df2[df2.source == source_map[x]]) == 0:
            return '+100%'
        else:
            return sign(round((len(df1[df1.source == source_map[x]])/len(df2[df2.source == source_map[x]])-1)*100))+'%'
    df7['Dynamics'] = df7['Source'].map(source_dynamics)
    return df7

In [ ]:
def name_png(table_name,customer):
    return str(table_name)+str(customer)+'.png'

In [ ]:
def save_png(df,table_name,customer):
    import matplotlib
    matplotlib.use('Agg') # must be before importing matplotlib.pyplot or pylab!
    import matplotlib.pyplot as plt
    from pandas.tools.plotting import table
    import numpy as np
    fig, ax = plt.subplots(figsize=(20, 5)) # set size frame
    ax.xaxis.set_visible(False)  # hide the x axis
    ax.yaxis.set_visible(False)  # hide the y axis
    ax.set_frame_on(False)  # no visible frame, uncomment if size is ok
    tabla = table(ax, df, loc='upper left', colWidths=[0.25,0.1,0.1,0.1])  # where df is your data frame
    tabla.auto_set_font_size(False) # activate set fontsize manually
    tabla.set_fontsize(12) # if ++fontsize is necessary ++colWidths
    tabla.scale(1.88, 2.5) # change size table
    plt.savefig(name_png(table_name,customer), transparent=True)

In [ ]:
df = df3_calculate(df1,df2)
save_png(df,'Count_of_records_','All')
df = df4_calculate(df1,df2)
save_png(df,'Processed_','All')
df = df7_calculate(df1,df2)
save_png(df,'Source_','All')
i=1
while i<=len(customers):
    df1cust = df1[df1.customer_id == i+1]
    df2cust = df2[df2.customer_id == i+1]
    customer = traslit(customers[i])
    df = df5_calculate(df1cust,df2cust,customer)
    save_png(df,'Sentiment_',customer)
    df = df6_calculate(df1cust,customer)
    save_png(df,'Objects_',customer)
    i=i+1